# Spatial fidelity template
Infers a spatial fiedlity map for every individual from a ```.mymridon``` experiment file and saves it as a csv, which can be further analyzed in the optinal part or exported into other software such as RStudio.  
This notebook uses the following:
* the py-myrmidon library ([Documentation](https://formicidae-tracker.github.io/myrmidon/latest/))
* scipy spatial library ([Documentation](https://docs.scipy.org/doc/scipy/reference/spatial.html))

In [1]:
from datetime import datetime, timedelta  # For convenient handling of time and date

import numpy as np  # Fundamental math library in python.
import pandas as pd  # To create a pandas dataframe, an equivalent to an R dataframe
import py_fort_myrmidon as fm
from scipy.spatial import KDTree

## Calculating spatial fidelity

The code here is the same as in the original code written by Matthias. For convenience sake the different components have been converted into functions that can be used to run over multiple fort-studio experiments

### Defining the grid for the spatial fidelity

A spatial tesselation of hexagons is used to define the sites where the observations of the individuals are counted. A nearest neighbour tree (KDTree) is used with the centers of the hexgons mentioned. For a given set of coordinates, a KDTree can be used to efficiently find the seed that is closest to the input coordinates, here the waypoints of the trajectory. The hexagonal boundaries are not defined explicitly. Simply by placing the hexagonal centers accordingly (see ```method ``` variable), the resulting pattern will be hexagonal or squared. The code in the following cell computes the hexagon centers that seed the KDTree.  
This part should probably only be edited w.r.t. the method and the internal radius.

In [2]:
def define_kdtree(method, xlim, ylim, r_int):
    if method == "vertical":
        dx = 2 * r_int
        dy = np.sqrt(3) * r_int
    elif method == "horizontal":
        dx = np.sqrt(3) * r_int
        dy = 2 * r_int
    elif method == "square":
        dx = 2 * r_int
        dy = 2 * r_int

    n_x = np.diff(xlim) // dx + 1
    n_y = np.diff(ylim) // dy + 1
    osx = n_x * dx - np.diff(xlim)
    osy = n_y * dy - np.diff(ylim)
    x = np.arange(start=xlim[0] - osx / 2, step=dx, stop=n_x * dx + osx / 2)
    y = np.arange(start=ylim[0] - osy / 2, step=dy, stop=n_y * dy + osy / 2)
    xm, ym = np.meshgrid(x, y)
    if method == "vertical":
        xm[::2] = xm[::2] + r_int
    elif method == "horizontal":
        ym[:, 1::2] = ym[:, 1::2] + r_int

    sites = np.stack([xm.flatten(), ym.flatten()], axis=1)
    kdtree = KDTree(sites)
    return kdtree, sites

### Function to calculate presence in the grid

This function takes the counts of each antID in each of the grid cells that were calculated using the kdtree function

In [3]:
def calc_counts_kdtree(exp, t_start, t_end, kdtree, sites, counts_cutoff, col_phase):
    """Function to calculate normalised counts of ants in each zone

    Args:
        exp (fort-myrmidon experiment file): Fort myrmidon experiment file
        t_start (datetime): Start time in local time with timezone set to None
        t_end (datetime): End time in local time with timezone set to None
        kdtree (kdtree object): kdtree object ontained from define_kdtree function
        counts (Numpy Array): A numpy array with dimentsions number of ants x number of zones
        idxmap (dict): Dictionary mapping ant ids to indices in the counts array
        counts_cutoff (int): cut-off value to threshold the counts
        col_phase (string): String which contains the colony and phase information
    """
    # Convert to fm.Time object
    t_st = fm.Time(t_start)
    t_en = fm.Time(t_end)
    # Extract ant trajectories
    trajectories = fm.Query.ComputeAntTrajectories(
        exp,
        start=t_st,
        end=t_en,
    )
    # Get unique ant IDs from trajectories
    trajectory_ant_ids = set()
    for t in trajectories:
        trajectory_ant_ids.add(t.Ant)
    trajectory_ant_ids = list(trajectory_ant_ids)
    # Create mapping from trajectory ant IDs to array indices
    idxmap = dict(zip(trajectory_ant_ids, range(len(trajectory_ant_ids))))
    print(f"Number of unique ants in trajectory data: {len(trajectory_ant_ids)}")
    # Create counts dictionary with proper dimensions
    counts = {}
    for s in exp.Spaces:
        counts[s] = np.zeros((len(trajectory_ant_ids), len(sites)))
    # Obtain the zone indices for each coordinate in each trajectory
    for t in trajectories:
        dist, zone_indices = kdtree.query(t.Positions[:, 1:3])
        ind, cts = np.unique(zone_indices, return_counts=True)
        counts[t.Space][idxmap[t.Ant], ind] += cts
    # Normalize the counts and prepare for merging
    space_dfs = {}  # Store dataframes by space name

    for s in exp.Spaces:
        space_name = exp.Spaces[s].Name.title()  # Get space name in title case
        print(f"Processing {space_name}")

        row_sums = counts[s].sum(axis=1)[:, np.newaxis]
        counts[s] = np.divide(
            counts[s], row_sums, out=np.zeros_like(counts[s]), where=row_sums != 0
        )

        # Create dataframe with renamed columns to include space name
        df = pd.DataFrame(data=counts[s], index=trajectory_ant_ids)
        df.index.name = "AntID"

        # Rename columns to format: 0_Nest, 1_Nest, etc.
        df.columns = [f"{i}_{space_name}" for i in range(df.shape[1])]

        # Store dataframe by space name
        space_dfs[space_name] = df

    # Sort space names to ensure Nest comes first, then Forage
    ordered_spaces = sorted(space_dfs.keys())
    if "Nest" in ordered_spaces:
        ordered_spaces.remove("Nest")
        ordered_spaces.insert(0, "Nest")
    # print(ordered_spaces)

    # Merge all dataframes using outer join to keep all ant IDs
    merged_df = None
    for space in ordered_spaces:
        if merged_df is None:
            merged_df = space_dfs[space]
            # print(merged_df.shape)
        else:
            # Join on index (AntID)
            merged_df = merged_df.join(space_dfs[space], how="outer")
            # print(merged_df.shape)
    # Create file name
    f_count = "NormCounts_{}_{}_{}.csv".format(
        col_phase,
        t_start.strftime("%Y%m%d-%H%M%S"),
        t_end.strftime("%Y%m%d-%H%M%S"),
    )  # Name for the output file
    print(f_count, " done")
    merged_df.to_csv(f_count)

### Function to run the spatial presence calculation over whole experiments

This is a wrapper function for `calc_counts_kdtree` which runs the function over multiple time ranges

In [4]:
def calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
):
    # Open myrmidon file
    exp = fm.Experiment.Open(myrmidon_path)
    # Get frames from first second of experiment tracking
    identifiedFrames = fm.Query.IdentifyFrames(
        exp,
        start=fm.Time(exp_start),
        end=fm.Time(exp_start).Add(fm.Duration().Parse("1s")),
    )
    # Obtain x and y limits fo frame
    xlim = [0, identifiedFrames[0].Width]
    ylim = [0, identifiedFrames[0].Height]
    # get average ant size to calculate the bin size
    # ant_radius = []
    # for ant in exp.Ants:
    #     ant_radius.append(fm.Query.ComputeMeasurementFor(experiment=exp, antID=ant, measurementTypeID=1)[0].LengthPixel)
    # r_int = np.mean(ant_radius)
    r_int = 300  # Using a general value of 300 to account for cases where the fm.Query.ComputeMeasurementFor throws an error
    # Obtain the kdtree
    kdtree, sites = define_kdtree("vertical", xlim, ylim, r_int)
    # Create a dictionary for holding counts data
    # counts_cutoff = 0 # For hard coding a count cutoff inside the function
    # Create a dictionary with an array full of zeros and shape (number individuals x number sites) for each Space
    # counts = {}
    # for s in exp.Spaces:
    #     counts[s] = np.zeros((len(exp.Ants), len(sites)))
    # idxmap = dict(
    #     zip(exp.Ants, range(len(exp.Ants)))
    # )  # Maps the ant id to the matrix index
    # Run function to calculate counts over each space for each combination of start and end time
    # Process each time period
    [
        calc_counts_kdtree(exp, t_start, t_end, kdtree, sites, counts_cutoff, col_phase)
        for t_start, t_end, col_phase in zip(
            start_time_list, end_time_list, col_phase_list
        )
    ]

## Colony Cfel42

In [ ]:
# Colony 42
myrmidon_path = "/media/ebiag/Ebi-2/Woundcare Experiment1/Cfell_wound_col42.myrmidon"
exp_start = datetime(2022, 4, 27, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 5, 1, 15, 54).astimezone(tz=None),
    datetime(2022, 5, 2, 16, 3).astimezone(tz=None),
    datetime(2022, 5, 3, 15, 53).astimezone(tz=None),
    datetime(2022, 5, 4, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 5, 15, 50).astimezone(tz=None),
    datetime(2022, 5, 6, 15, 55).astimezone(tz=None),
    datetime(2022, 5, 2, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 3, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 4, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 5, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 6, 9, 0).astimezone(tz=None),
    datetime(2022, 5, 7, 9, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel42_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

In [ ]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    [datetime(2022, 4, 27, 0, 1).astimezone(tz=None)],
    [datetime(2022, 5, 1, 23, 59).astimezone(tz=None)],
    counts_cutoff,
    ["Cfel42_Baseline"],
)

## Colony Cfel1

In [ ]:
# Colony 1
myrmidon_path = "/media/ebiag/Ebi-2/Woundcare Experiment2/woundcare_cfell1_T2.myrmidon"
exp_start = datetime(2022, 5, 31, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 6, 4, 14, 48).astimezone(tz=None),
    datetime(2022, 6, 5, 14, 57).astimezone(tz=None),
    datetime(2022, 6, 6, 14, 30).astimezone(tz=None),
    datetime(2022, 6, 7, 14, 49).astimezone(tz=None),
    datetime(2022, 6, 8, 14, 43).astimezone(tz=None),
    datetime(2022, 6, 9, 15, 5).astimezone(tz=None),
    datetime(2022, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 6, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 7, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 8, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 9, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 10, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel1_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

In [ ]:
# Comment out the normalisation in the calc_spatial_fidelity function to obtain the raw counts

calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    [
        datetime(2022, 5, 31, 0, 1).astimezone(tz=None),
        datetime(2022, 6, 4, 6, 0).astimezone(tz=None),
    ],
    [
        datetime(2022, 6, 2, 23, 59).astimezone(tz=None),
        datetime(2022, 6, 4, 23, 59).astimezone(tz=None),
    ],
    counts_cutoff,
    ["Cfel1_Baseline1", "Cfel1_Baseline2"],
)

In [ ]:
b1 = pd.read_csv("NormCounts_Cfel1_Baseline1_20220531-000100_20220602-235900.csv")
b2 = pd.read_csv("NormCounts_Cfel1_Baseline2_20220604-060000_20220604-235900.csv")

# Set AntID as index
b1.set_index("AntID", inplace=True)
b2.set_index("AntID", inplace=True)

# Get the original column order from b1
original_columns = b1.columns.tolist()

# Create a new combined dataframe with all ants and the original columns
combined_index = sorted(list(set(b1.index) | set(b2.index)))
result = pd.DataFrame(0, index=combined_index, columns=original_columns)

# Add data from both datasets
for col in original_columns:
    if col in b1.columns:
        result.loc[b1.index, col] += b1[col]

    if col in b2.columns:
        result.loc[b2.index, col] += b2[col]

# Split columns by type while maintaining order
nest_cols = [col for col in original_columns if col.endswith("_Nest")]
forage_cols = [col for col in original_columns if col.endswith("_Forage")]

# Normalize Nest columns separately
if nest_cols:
    nest_sum = result[nest_cols].sum(axis=1)
    for col in nest_cols:
        result[col] = result[col].div(nest_sum, axis=0).fillna(0)

# Normalize Forage columns separately
if forage_cols:
    forage_sum = result[forage_cols].sum(axis=1)
    for col in forage_cols:
        result[col] = result[col].div(forage_sum, axis=0).fillna(0)

# Check row sums
# Create summary dataframe from the results
# This will have AntIDs as rows and two columns: Nest and Forage
summary_df = pd.DataFrame(index=result.index)

# Sum all Nest columns for each ant
if nest_cols:
    summary_df["Nest"] = result[nest_cols].sum(axis=1)
else:
    summary_df["Nest"] = 0

# Sum all Forage columns for each ant
if forage_cols:
    summary_df["Forage"] = result[forage_cols].sum(axis=1)
else:
    summary_df["Forage"] = 0

# Display the first few rows to verify
# print(summary_df.head())


# Save the result
result.to_csv("NormCounts_Cfel1_Baseline_20220531-000100_20220604-235900.csv")

## Colony Cfel54

In [5]:
myrmidon_path = "/media/ebiag/Ebi-2/Woundcare Experiment3/woundcare_cfell54_T3.myrmidon"
exp_start = datetime(2022, 6, 15, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2022, 6, 19, 14, 26).astimezone(tz=None),
    datetime(2022, 6, 20, 14, 35).astimezone(tz=None),
    datetime(2022, 6, 21, 14, 21).astimezone(tz=None),
    datetime(2022, 6, 22, 14, 28).astimezone(tz=None),
    datetime(2022, 6, 23, 14, 14).astimezone(tz=None),
    datetime(2022, 6, 24, 14, 31).astimezone(tz=None),
    datetime(2022, 6, 20, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 21, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 22, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 23, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 24, 8, 0).astimezone(tz=None),
    datetime(2022, 6, 25, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel54_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [6]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

Computing ant trajectories: 100%|████| 360/360 [00:06<00:00, 51.89tracked min/s]


Number of unique ants in trajectory data: 86
Processing Nest
Processing Forage
NormCounts_Cfel54_Control_20220619-142600_20220619-202600.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:03<00:00, 92.20tracked min/s]


Number of unique ants in trajectory data: 84
Processing Nest
Processing Forage
NormCounts_Cfel54_R1_20220620-143500_20220620-203500.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:03<00:00, 94.43tracked min/s]


...

Number of unique ants in trajectory data: 83
Processing Nest
Processing Forage
NormCounts_Cfel54_PostR3_20220623-080000_20220623-140000.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:03<00:00, 93.23tracked min/s]


Number of unique ants in trajectory data: 80
Processing Nest
Processing Forage
NormCounts_Cfel54_PostR4_20220624-080000_20220624-140000.csv  done


Computing ant trajectories: 100%|███| 360/360 [00:03<00:00, 105.24tracked min/s]


Number of unique ants in trajectory data: 77
Processing Nest
Processing Forage
NormCounts_Cfel54_PostR5_20220625-080000_20220625-140000.csv  done


In [7]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    [datetime(2022, 6, 15, 0, 1).astimezone(tz=None)],
    [datetime(2022, 6, 19, 23, 59).astimezone(tz=None)],
    counts_cutoff,
    ["Cfel54_Baseline"],
)

Computing ant trajectories: 100%|█▉| 7197/7198 [03:39<00:00, 32.82tracked min/s]


Number of unique ants in trajectory data: 99
Processing Nest
Processing Forage
NormCounts_Cfel54_Baseline_20220615-000100_20220619-235900.csv  done


## Colony Cfel 55

In [12]:
# Colony 55
myrmidon_path = "/media/ebiag/Ebi-1/InfectionExp_Cfel55/InfectionExpCol55.myrmidon"
exp_start = datetime(2023, 4, 14, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2023, 4, 18, 14, 40).astimezone(tz=None),
    datetime(2023, 4, 20, 15, 45).astimezone(tz=None),
    datetime(2023, 4, 21, 14, 48).astimezone(tz=None),
    datetime(2023, 4, 22, 14, 17).astimezone(tz=None),
    datetime(2023, 4, 23, 14, 0).astimezone(tz=None),
    datetime(2023, 4, 24, 14, 54).astimezone(tz=None),
    datetime(2023, 4, 20, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 21, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 22, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 23, 7, 30).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel55_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [13]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

Computing ant trajectories: 100%|███▉| 359/360 [00:05<00:00, 65.08tracked min/s]


Number of unique ants in trajectory data: 86
Processing Forage
Processing Nest
NormCounts_Cfel55_Control_20230418-144000_20230418-204000.csv  done


Computing ant trajectories: 100%|████| 360/360 [00:04<00:00, 72.04tracked min/s]


Number of unique ants in trajectory data: 79
Processing Forage
Processing Nest
NormCounts_Cfel55_R1_20230420-154500_20230420-214500.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:05<00:00, 66.20tracked min/s]


...

Number of unique ants in trajectory data: 74
Processing Forage
Processing Nest
NormCounts_Cfel55_PostR3_20230423-073000_20230423-133000.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 82.05tracked min/s]


Number of unique ants in trajectory data: 71
Processing Forage
Processing Nest
NormCounts_Cfel55_PostR4_20230424-080000_20230424-140000.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 78.30tracked min/s]


Number of unique ants in trajectory data: 71
Processing Forage
Processing Nest
NormCounts_Cfel55_PostR5_20230425-080000_20230425-140000.csv  done


In [14]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    [datetime(2023, 4, 14, 0, 1).astimezone(tz=None)],
    [datetime(2023, 4, 18, 23, 59).astimezone(tz=None)],
    counts_cutoff,
    ["Cfel55_Baseline"],
)

Computing ant trajectories: 100%|█▉| 7197/7198 [01:32<00:00, 77.86tracked min/s]


Number of unique ants in trajectory data: 102
Processing Forage
Processing Nest
NormCounts_Cfel55_Baseline_20230414-000100_20230418-235900.csv  done


## Colony Cfel 13

In [15]:
# Colony 13
myrmidon_path = "/media/ebiag/Ebi-4/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
exp_start = datetime(2023, 4, 19, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2023, 4, 23, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 24, 15, 29).astimezone(tz=None),
    datetime(2023, 4, 25, 14, 19).astimezone(tz=None),
    datetime(2023, 4, 26, 15, 3).astimezone(tz=None),
    datetime(2023, 4, 27, 16, 43).astimezone(tz=None),
    datetime(2023, 4, 28, 14, 27).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 26, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 27, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 28, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 29, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel13_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [16]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 76.95tracked min/s]


Number of unique ants in trajectory data: 109
Processing Nest
Processing Forage
NormCounts_Cfel13_Control_20230423-150500_20230423-210500.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 75.12tracked min/s]


Number of unique ants in trajectory data: 108
Processing Nest
Processing Forage
NormCounts_Cfel13_R1_20230424-152900_20230424-212900.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 73.10tracked min/s]


...

Number of unique ants in trajectory data: 97
Processing Nest
Processing Forage
NormCounts_Cfel13_PostR3_20230427-080000_20230427-140000.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 74.28tracked min/s]


Number of unique ants in trajectory data: 96
Processing Nest
Processing Forage
NormCounts_Cfel13_PostR4_20230428-080000_20230428-140000.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 82.02tracked min/s]


Number of unique ants in trajectory data: 92
Processing Nest
Processing Forage
NormCounts_Cfel13_PostR5_20230429-080000_20230429-140000.csv  done


In [17]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    [datetime(2023, 4, 19, 0, 1).astimezone(tz=None)],
    [datetime(2023, 4, 23, 23, 59).astimezone(tz=None)],
    counts_cutoff,
    ["Cfel13_Baseline"],
)

Computing ant trajectories: 100%|█▉| 7197/7198 [01:36<00:00, 74.49tracked min/s]


Number of unique ants in trajectory data: 119
Processing Nest
Processing Forage
NormCounts_Cfel13_Baseline_20230419-000100_20230423-235900.csv  done


## Colony Cfel 64

In [18]:
# Colony 64
myrmidon_path = "/media/ebiag/Ebi-4/InfectionExp_Cfel64/InfectionExpCol64.myrmidon"
exp_start = datetime(2023, 5, 27, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2023, 5, 31, 15, 5).astimezone(tz=None),
    datetime(2023, 6, 1, 15, 51).astimezone(tz=None),
    datetime(2023, 6, 2, 14, 44).astimezone(tz=None),
    datetime(2023, 6, 3, 14, 50).astimezone(tz=None),
    datetime(2023, 6, 4, 14, 43).astimezone(tz=None),
    datetime(2023, 6, 5, 14, 52).astimezone(tz=None),
    datetime(2023, 6, 1, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 2, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 3, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 4, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 5, 8, 0).astimezone(tz=None),
    datetime(2023, 6, 6, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel64_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [19]:
c64_list = calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    start_time_list,
    end_time_list,
    counts_cutoff,
    col_phase_list,
)

Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 80.62tracked min/s]


Number of unique ants in trajectory data: 103
Processing Forage
Processing Nest
NormCounts_Cfel64_Control_20230531-150500_20230531-210500.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 76.26tracked min/s]


Number of unique ants in trajectory data: 103
Processing Forage
Processing Nest
NormCounts_Cfel64_R1_20230601-155100_20230601-215100.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 82.22tracked min/s]


...

Number of unique ants in trajectory data: 101
Processing Forage
Processing Nest
NormCounts_Cfel64_PostR3_20230604-080000_20230604-140000.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 79.07tracked min/s]


Number of unique ants in trajectory data: 97
Processing Forage
Processing Nest
NormCounts_Cfel64_PostR4_20230605-080000_20230605-140000.csv  done


Computing ant trajectories: 100%|███▉| 359/360 [00:04<00:00, 80.87tracked min/s]


Number of unique ants in trajectory data: 94
Processing Forage
Processing Nest
NormCounts_Cfel64_PostR5_20230606-080000_20230606-140000.csv  done


In [20]:
calc_spatial_fidelity(
    myrmidon_path,
    exp_start,
    [datetime(2023, 5, 27, 0, 1).astimezone(tz=None)],
    [datetime(2023, 5, 31, 23, 59).astimezone(tz=None)],
    counts_cutoff,
    ["Cfel64_Baseline"],
)

Computing ant trajectories: 100%|█▉| 7197/7198 [01:27<00:00, 81.91tracked min/s]


Number of unique ants in trajectory data: 115
Processing Forage
Processing Nest
NormCounts_Cfel64_Baseline_20230527-000100_20230531-235900.csv  done


### Verifying code

In [ ]:
# Colony 13
myrmidon_path = "/media/ebiag/Ebi-4/InfectionExp_Cfel13/InfectionExp_Cfel13.myrmidon"
exp_start = datetime(2023, 4, 19, 0, 1).astimezone(tz=None)
start_time_list = [
    datetime(2023, 4, 23, 15, 5).astimezone(tz=None),
    datetime(2023, 4, 24, 15, 29).astimezone(tz=None),
    datetime(2023, 4, 25, 14, 19).astimezone(tz=None),
    datetime(2023, 4, 26, 15, 3).astimezone(tz=None),
    datetime(2023, 4, 27, 16, 43).astimezone(tz=None),
    datetime(2023, 4, 28, 14, 27).astimezone(tz=None),
    datetime(2023, 4, 24, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 25, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 26, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 27, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 28, 8, 0).astimezone(tz=None),
    datetime(2023, 4, 29, 8, 0).astimezone(tz=None),
]

counts_cutoff = 0

end_time_list = [start_time + timedelta(hours=6) for start_time in start_time_list]

colony = "Cfel13_"
phase = [
    "Control",
    "R1",
    "R2",
    "R3",
    "R4",
    "R5",
    "PostC",
    "PostR1",
    "PostR2",
    "PostR3",
    "PostR4",
    "PostR5",
]

col_phase_list = [f"{colony}{phase}" for phase in phase]

In [ ]:
# Open myrmidon file
exp = fm.Experiment.Open(myrmidon_path)
# Get frames from first second of experiment tracking
identifiedFrames = fm.Query.IdentifyFrames(
    exp,
    start=fm.Time(exp_start),
    end=fm.Time(exp_start).Add(fm.Duration().Parse("1s")),
)
# Obtain x and y limits fo frame
xlim = [0, identifiedFrames[0].Width]
ylim = [0, identifiedFrames[0].Height]

r_int = 300  # Using a general value of 300 to account for cases where the fm.Query.ComputeMeasurementFor throws an error
# Obtain the kdtree
kdtree, sites = define_kdtree("vertical", xlim, ylim, r_int)

In [ ]:
# Convert to fm.Time object
t_st = fm.Time(start_time_list[10])
t_en = fm.Time(end_time_list[10])

In [ ]:
# Extract ant trajectories
trajectories = fm.Query.ComputeAntTrajectories(
    exp,
    start=t_st,
    end=t_en,
)

In [ ]:
# Get unique ant IDs from trajectories
trajectory_ant_ids = set()
for t in trajectories:
    trajectory_ant_ids.add(t.Ant)
trajectory_ant_ids = list(trajectory_ant_ids)
len(trajectory_ant_ids)

In [ ]:
# Create mapping from trajectory ant IDs to array indices
idxmap = dict(zip(trajectory_ant_ids, range(len(trajectory_ant_ids))))
idxmap

In [ ]:
# Create counts dictionary with proper dimensions
counts = {}
for s in exp.Spaces:
    counts[s] = np.zeros((len(trajectory_ant_ids), len(sites)))

In [ ]:
# Obtain the zone indices for each coordinate in each trajectory
for t in trajectories:
    dist, zone_indices = kdtree.query(t.Positions[:, 1:3])
    ind, cts = np.unique(zone_indices, return_counts=True)
    counts[t.Space][idxmap[t.Ant], ind] += cts

In [ ]:
# Normalize the counts
for s in exp.Spaces:
    print(exp.Spaces[s].Name)
    row_sums = counts[s].sum(axis=1)[:, np.newaxis]
    counts[s] = np.divide(
        counts[s], row_sums, out=np.zeros_like(counts[s]), where=row_sums != 0
    )

    # Output the counts to a csv file
    df = pd.DataFrame(data=counts[s], index=trajectory_ant_ids)
    df.index.name = "AntID"

In [ ]:
# Normalize the counts and prepare for merging
space_dfs = {}  # Store dataframes by space name

for s in exp.Spaces:
    space_name = exp.Spaces[s].Name.title()  # Get space name in title case
    print(f"Processing {space_name}")

    row_sums = counts[s].sum(axis=1)[:, np.newaxis]
    counts[s] = np.divide(
        counts[s], row_sums, out=np.zeros_like(counts[s]), where=row_sums != 0
    )

    # Create dataframe with renamed columns to include space name
    df = pd.DataFrame(data=counts[s], index=trajectory_ant_ids)
    df.index.name = "AntID"

    # Rename columns to format: 0_Nest, 1_Nest, etc.
    df.columns = [f"{i}_{space_name}" for i in range(df.shape[1])]

    # Store dataframe by space name
    space_dfs[space_name] = df

In [ ]:
df1 = space_dfs["Nest"]
df2 = space_dfs["Forage"]
print(df1.shape, df2.shape)

In [ ]:
# Identify rows which are completely 0 in df1
zero_rows = df1.index[df1.sum(axis=1) == 0]
print(f"Rows with all 0s in Nest: {zero_rows}")

In [ ]:
# Same for df2
zero_rows = df2.index[df2.sum(axis=1) == 0]
print(f"Rows with all 0s in Forage: {zero_rows}")

In [ ]:
# Sort space names to ensure Nest comes first, then others alphabetically
ordered_spaces = sorted(space_dfs.keys())
ordered_spaces

In [ ]:
if "Nest" in ordered_spaces:
    ordered_spaces.remove("Nest")
    ordered_spaces.insert(0, "Nest")
ordered_spaces

In [ ]:
# Merge all dataframes using outer join to keep all ant IDs
merged_df = None
for space in ordered_spaces:
    if merged_df is None:
        merged_df = space_dfs[space]
    else:
        # Join on index (AntID)
        merged_df = merged_df.join(space_dfs[space], how="outer")

In [ ]:
# Identify rows with 0 values in all columns in merged_df
zero_rows = merged_df.index[merged_df.sum(axis=1) == 0]
print(f"Rows with all 0s in all spaces: {zero_rows}")

# Identify rows with NaN values in all columns in merged_df
nan_rows = merged_df.index[merged_df.isna().all(axis=1)]
print(f"Rows with NaNs in all spaces: {nan_rows}")

### Verifying output

In [ ]:
import pandas as pd
import glob
import os

# 1. Identify all files with "NormCounts" in the name
norm_count_files = glob.glob("*NormCounts*.csv")
print(f"Found {len(norm_count_files)} NormCounts files")

# 2-4. Load each file and check column count
expected_columns = 391  # 1 AntID + 195 Nest + 195 Forage columns
invalid_files = []

for file in norm_count_files:
    # Load the file
    df = pd.read_csv(file)

    # Check column count
    if len(df.columns) != expected_columns:
        invalid_files.append({"filename": file, "columns": len(df.columns)})

# Print results
if invalid_files:
    print("\nFiles with incorrect column count:")
    for file_info in invalid_files:
        print(
            f"- {file_info['filename']}: {file_info['columns']} columns (expected {expected_columns})"
        )
    print(f"\nTotal invalid files: {len(invalid_files)}")
else:
    print("\nAll files have the correct number of columns!")

Found 78 NormCounts files

All files have the correct number of columns!
